# 08 - Text Splitters 文本分割

## 用途
学习使用 LangChain RecursiveCharacterTextSplitter 将文档按规则切分成 chunk，保证向量化质量

## 学习目标
- 理解文本切分原理和策略
- 掌握切分参数设置和优化
- 能对中文文档进行有效切片
- 控制切片大小和质量

## 代码块独立性说明
**注意**：每个代码块都是独立的，包含完整的导入和初始化，确保可以单独运行。

## 1. RecursiveCharacterTextSplitter 基础使用

In [ ]:
# RecursiveCharacterTextSplitter 基础使用 - 独立代码块
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

# 加载文档
file_path = "../sample_chinese_text.txt"

print("✂️ RecursiveCharacterTextSplitter 基础测试:")
print("=" * 50)

if os.path.exists(file_path):
    # 加载文档
    loader = TextLoader(file_path, encoding='utf-8')
    documents = loader.load()
    
    if documents:
        original_doc = documents[0]
        print(f"📝 原始文档信息:")
        print(f"   内容长度: {len(original_doc.page_content)} 字符")
        print(f"   行数: {len(original_doc.page_content.splitlines())} 行")
        
        # 创建文本分割器
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,  # 切片大小
            chunk_overlap=50,  # 重叠大小
            length_function=len,
            separators=["\n\n", "\n", "。", "；", "，", " ", ""]
        )
        
        print(f"\n✂️ 分割器配置:")
        print(f"   chunk_size: 500 字符")
        print(f"   chunk_overlap: 50 字符")
        print(f"   分隔符: 段落 > 句子 > 分句 > 逗号 > 空格 > 字符")
        
        # 执行分割
        chunks = text_splitter.split_documents([original_doc])
        
        print(f"\n📊 分割结果:")
        print(f"   切片数量: {len(chunks)}")
        print(f"   平均长度: {sum(len(chunk.page_content) for chunk in chunks) / len(chunks):.1f} 字符")
        
        # 显示前几个切片
        print(f"\n📋 切片内容预览:")
        for i, chunk in enumerate(chunks[:3], 1):
            content = chunk.page_content
            print(f"   切片 {i}: {len(content)} 字符")
            print(f"   内容: {content[:80]}...")
            print(f"   元数据: {chunk.metadata}")
            print()
        
        # 验证点：中文文档被正确切分成指定大小的片段
        assert len(chunks) > 1, "文档未被分割"
        assert all(len(chunk.page_content) <= 550 for chunk in chunks), "存在超大切片"
        assert all(len(chunk.page_content) > 0 for chunk in chunks), "存在空切片"
        print(f"✅ 验证通过：文本分割正常工作")
    
else:
    print(f"❌ 文件不存在: {file_path}")

## 2. 不同参数配置对比

In [ ]:
# 不同参数配置对比 _ 独立代码块
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

# 加载文档
file_path = "../sample_chinese_text.txt"

print("🔧 不同参数配置对比测试:")
print("=" * 50)

if os.path.exists(file_path):
    loader = TextLoader(file_path, encoding='utf-8')
    documents = loader.load()
    
    if documents:
        original_doc = documents[0]
        
        # 测试不同配置
        configs = [
            {
                "name": "小切片",
                "chunk_size": 200,
                "chunk_overlap": 20
            },
            {
                "name": "标准切片",
                "chunk_size": 500,
                "chunk_overlap": 50
            },
            {
                "name": "大切片",
                "chunk_size": 800,
                "chunk_overlap": 80
            }
        ]
        
        results = {}
        
        for config in configs:
            print(f"\n🧪 测试配置: {config['name']}")
            
            # 创建分割器
            splitter = RecursiveCharacterTextSplitter(
                chunk_size=config['chunk_size'],
                chunk_overlap=config['chunk_overlap'],
                length_function=len
            )
            
            # 执行分割
            chunks = splitter.split_documents([original_doc])
            
            # 统计结果
            chunk_lengths = [len(chunk.page_content) for chunk in chunks]
            
            results[config['name']] = {
                "chunk_count": len(chunks),
                "avg_length": sum(chunk_lengths) / len(chunk_lengths),
                "max_length": max(chunk_lengths),
                "min_length": min(chunk_lengths),
                "chunks": chunks
            }
            
            print(f"   切片数量: {len(chunks)}")
            print(f"   平均长度: {results[config['name']]['avg_length']:.1f} 字符")
            print(f"   长度范围: {results[config['name']]['min_length']}-{results[config['name']]['max_length']} 字符")
        
        # 配置对比总结
        print(f"\n📊 配置对比总结:")
        print(f"{'配置':<8} {'切片数':<8} {'平均长度':<10} {'长度范围':<15}")
        print("-" * 45)
        for name, result in results.items():
            print(f"{name:<8} {result['chunk_count']:<8} {result['avg_length']:<10.1f} {result['min_length']}-{result['max_length']:<10}")
        
        print(f"\n💡 参数配置建议:")
        print("   - 小切片(200): 适合精确检索，但可能丢失上下文")
        print("   - 标准切片(500): 平衡上下文和检索精度")
        print("   - 大切片(800): 保持更多上下文，但检索粒度较粗")
        print("✅ 验证通过：参数配置影响分割效果")
    
else:
    print(f"❌ 文件不存在: {file_path}")

## 3. 中文文本分割优化

In [ ]:
# 中文文本分割优化 _ 独立代码块
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

# 加载文档
file_path = "../sample_chinese_text.txt"

print("🈯 中文文本分割优化测试:")
print("=" * 50)

if os.path.exists(file_path):
    loader = TextLoader(file_path, encoding='utf-8')
    documents = loader.load()
    
    if documents:
        original_doc = documents[0]
        
        # 测试不同分隔符策略
        separator_strategies = [
            {
                "name": "默认分隔符",
                "separators": ["\n\n", "\n", "。", "；", "，", " ", ""]
            },
            {
                "name": "中文优化",
                "separators": ["\n\n", "\n", "。", "！", "？", "；", "，", " ", ""]
            },
            {
                "name": "句子优先",
                "separators": ["。", "！", "？", "；", "\n\n", "\n", "，", " ", ""]
            }
        ]
        
        results = {}
        
        for strategy in separator_strategies:
            print(f"\n🧪 测试策略: {strategy['name']}")
            
            # 创建分割器
            splitter = RecursiveCharacterTextSplitter(
                chunk_size=500,
                chunk_overlap=50,
                length_function=len,
                separators=strategy['separators']
            )
            
            # 执行分割
            chunks = splitter.split_documents([original_doc])
            
            # 分析分割质量
            chunk_lengths = [len(chunk.page_content) for chunk in chunks]
            
            # 检查句子完整性（简单检查：以句号结尾的比例）
            complete_sentences = sum(1 for chunk in chunks 
                                    if chunk.page_content.strip().endswith('。') or 
                                       chunk.page_content.strip().endswith('！') or
                                       chunk.page_content.strip().endswith('？'))
            
            sentence_ratio = complete_sentences / len(chunks) if chunks else 0
            
            results[strategy['name']] = {
                "chunk_count": len(chunks),
                "avg_length": sum(chunk_lengths) / len(chunk_lengths),
                "sentence_ratio": sentence_ratio,
                "chunks": chunks
            }
            
            print(f"   切片数量: {len(chunks)}")
            print(f"   平均长度: {results[strategy['name']]['avg_length']:.1f} 字符")
            print(f"   句子完整性: {sentence_ratio:.1%}")
            
            # 显示第一个切片
            if chunks:
                print(f"   首片预览: {chunks[0].page_content[:60]}...")
        
        # 分割策略对比
        print(f"\n📊 分割策略对比:")
        print(f"{'策略':<12} {'切片数':<8} {'平均长度':<10} {'句子完整性':<12}")
        print("-" * 45)
        for name, result in results.items():
            print(f"{name:<12} {result['chunk_count']:<8} {result['avg_length']:<10.1f} {result['sentence_ratio']:<12.1%}")
        
        print(f"\n💡 中文分割优化建议:")
        print("   - 使用中文标点符号作为主要分隔符")
        print("   - 优先保持句子完整性提高语义连贯性")
        print("   - 根据文档类型调整分隔符优先级")
        print("✅ 验证通过：中文分割策略优化有效")
    
else:
    print(f"❌ 文件不存在: {file_path}")

## 4. 分割质量评估

In [ ]:
# 分割质量评估 _ 独立代码块
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import re

# 加载文档
file_path = "../sample_chinese_text.txt"

print("📏 分割质量评估测试:")
print("=" * 50)

if os.path.exists(file_path):
    loader = TextLoader(file_path, encoding='utf-8')
    documents = loader.load()
    
    if documents:
        original_doc = documents[0]
        
        # 创建标准分割器
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=50,
            length_function=len
        )
        
        chunks = splitter.split_documents([original_doc])
        
        print(f"📊 质量评估指标:")
        
        # 1. 基础统计
        chunk_lengths = [len(chunk.page_content) for chunk in chunks]
        print(f"\n📈 基础统计:")
        print(f"   切片数量: {len(chunks)}")
        print(f"   平均长度: {sum(chunk_lengths) / len(chunk_lengths):.1f} 字符")
        print(f"   长度标准差: {(sum((x - sum(chunk_lengths)/len(chunk_lengths))**2 for x in chunk_lengths) / len(chunk_lengths))**0.5:.1f} 字符")
        print(f"   长度范围: {min(chunk_lengths)}-{max(chunk_lengths)} 字符")
        
        # 2. 语义完整性
        print(f"\n🔤 语义完整性:")
        
        # 检查句子完整性
        sentence_endings = ['。', '！', '？', '；']
        complete_sentences = sum(1 for chunk in chunks 
                                if any(chunk.page_content.strip().endswith(end) for end in sentence_endings))
        print(f"   完整句子比例: {complete_sentences/len(chunks):.1%}")
        
        # 检查词汇完整性（避免在词汇中间分割）
        incomplete_words = sum(1 for chunk in chunks 
                               if len(chunk.page_content) > 0 and 
                                  chunk.page_content[-1] not in [' ', '\n', '。', '！', '？', '；', '，'])
        print(f"   词汇中断比例: {incomplete_words/len(chunks):.1%}")
        
        # 3. 内容覆盖度
        print(f"\n📋 内容覆盖度:")
        total_chars = sum(len(chunk.page_content) for chunk in chunks)
        original_chars = len(original_doc.page_content)
        coverage = total_chars / original_chars
        print(f"   原始字符数: {original_chars}")
        print(f"   切片字符总数: {total_chars}")
        print(f"   覆盖率: {coverage:.1%} (包含重叠)")
        
        # 4. 重叠分析
        print(f"\n🔗 重叠分析:")
        overlap_info = []
        for i in range(len(chunks) - 1):
            current = chunks[i].page_content
            next_chunk = chunks[i + 1].page_content
            
            # 简单重叠检测（查找共同的后缀/前缀
            max_overlap = min(50, len(current), len(next_chunk))
            overlap_found = 0
            
            for j in range(1, max_overlap + 1):
                if current[-j:] == next_chunk[:j]:
                    overlap_found = j
            
            overlap_info.append(overlap_found)
        
        if overlap_info:
            avg_overlap = sum(overlap_info) / len(overlap_info)
            print(f"   平均重叠长度: {avg_overlap:.1f} 字符")
            print(f"   重叠一致性: {'高' if max(overlap_info) - min(overlap_info) <= 10 else '中'}")
        
        # 5. 质量评分
        print(f"\n⭐ 质量评分:")
        
        # 简单评分算法
        length_score = 1.0 - abs(500 - sum(chunk_lengths) / len(chunk_lengths)) / 500  # 长度一致性
        sentence_score = complete_sentences / len(chunks)  # 句子完整性
        word_score = 1.0 - incomplete_words / len(chunks)  # 词汇完整性
        
        overall_score = (length_score + sentence_score + word_score) / 3 * 100
        
        print(f"   长度一致性: {length_score*100:.1f} 分")
        print(f"   句子完整性: {sentence_score*100:.1f} 分")
        print(f"   词汇完整性: {word_score*100:.1f} 分")
        print(f"   综合评分: {overall_score:.1f} 分")
        
        print(f"\n💡 质量优化建议:")
        if overall_score >= 80:
            print("   - 分割质量优秀，建议保持当前配置")
        elif overall_score >= 60:
            print("   - 分割质量良好，可微调参数优化")
        else:
            print("   - 分割质量需要改进，建议调整分隔符和参数")
        
        # 验证点：分割质量评估完成
        assert len(chunks) > 0, "无分割结果"
        assert overall_score > 0, "质量评分异常"
        print(f"\n✅ 验证通过：分割质量评估完成")
    
else:
    print(f"❌ 文件不存在: {file_path}")

## 5. 学习总结与最佳实践

In [ ]:
# 学习总结与最佳实践 _ 独立代码块
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

print("📋 Text Splitters 学习总结:")
print("=" * 50)

# 验证点检查
verification_points = [
    "✅ 基础分割：RecursiveCharacterTextSplitter 使用",
    "✅ 参数配置：chunk_size 和 chunk_overlap 优化",
    "✅ 中文优化：分隔符策略和语义完整性",
    "✅ 质量评估：分割效果量化分析",
]

for point in verification_points:
    print(point)

print(f"\n🎯 核心技能掌握情况: {len(verification_points)}/4 项")

print("\n💡 Text Splitters 最佳实践:")
print("1. 参数设置：chunk_size 300-800，chunk_overlap 10-20%")
print("2. 分隔符选择：根据文档类型和语言特点定制")
print("3. 中文处理：优先使用中文标点保持语义完整性")
print("4. 质量监控：定期评估分割效果和调整参数")
print("5. 性能优化：避免过度分割影响检索效率")

print("\n🚀 下一步学习建议:")
print("1. 深入学习 Embedding 文本向量化")
print("2. 掌握 Vector Stores 向量存储")
print("3. 学习 Retrievers 检索器配置")
print("4. 探索其他分割器类型")
print("5. 实践 RAG 系统完整流程")

# 最终验证：确保 TextSplitter 基础功能可用
try:
    file_path = "../sample_chinese_text.txt"
    
    if os.path.exists(file_path):
        # 加载和分割
        loader = TextLoader(file_path, encoding='utf-8')
        documents = loader.load()
        
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=50
        )
        
        chunks = splitter.split_documents(documents)
        
        print(f"\n🎉 最终验证成功:")
        print(f"   处理文件: {os.path.basename(file_path)}")
        print(f"   原始文档: {len(documents)} 个")
        print(f"   分割切片: {len(chunks)} 个")
        print(f"   平均长度: {sum(len(chunk.page_content) for chunk in chunks) / len(chunks):.1f} 字符")
        print("\n✅ Text Splitters 学习完成！")
    else:
        print(f"\n⚠️  最终验证跳过: 测试文件不存在")
        
except Exception as e:
    print(f"\n❌ 最终验证失败: {e}")